Only include accounts with <1000 follows

Mentions:
1) Gets tweets that mention the user. Retrieve most recent 100 tweets that mention user
- Client.get_users_mentions(twitter_id, max_results=100, pagination_token=next_page_token)
- 180 requests per 15-minute

In [20]:
import pandas as pd
import numpy as np
import networkx as nx
import json
import requests

In [21]:
import tweepy
from tweepy import Client

key = 'B81Sj4ifYRstLZGMyMDM0roG3'
secret = 'c1Rg4vdyl2mjMEC7WKDsVLTz2IlqTHy1yezn71Y9cw3LgZaxAW'
token = 'AAAAAAAAAAAAAAAAAAAAAD6eWQEAAAAABtT80i7lkSemhYsuOIjy4O4xLQs%3DkfsByhRoXQ9odyT51sECR1vXxOLKxdtOON2Ij4OODdAt8jTcGY'

# key='CSh1kvK1gpxzyhl8GYuMa0RIv'
# secret='yfgNtKNVeqdjqktu9lR4eK0oEt68KjXhVV8jJOJgvqrvpFbL9N'
# token='AAAAAAAAAAAAAAAAAAAAAOCeVAEAAAAABD8nomZEvbNVV3p7dFOVkhm5Kv8%3DritcqLa43rGxvmEUXtuSVp6fwDKsONk6hbgPVGyjmqI72ueB9W'

api = tweepy.Client(bearer_token=token, consumer_key=key, consumer_secret=secret, 
              return_type=requests.Response,
              wait_on_rate_limit=True)

In [22]:
import os

follows_processed = []
mentions_already_processed = []
 
for name in os.listdir('human'):
    if os.path.isdir(name):
        continue
    list_name = name.replace('.pkl','').replace('edgelist_2_deg_','')
    follows_processed.append(int(list_name))
    
    
for name in os.listdir('mentions/human'):
    if os.path.isdir(name):
        continue
    list_name = name.replace('.pkl','').replace('edgelist_mentions_','')
    mentions_already_processed.append(int(list_name))

In [23]:
follows_processed

[1039491917145432064,
 1047707161571328000,
 1012481676524576770,
 998977009056444416,
 998242564422750209,
 884092539418152960,
 1914493741,
 743063090183340032,
 849258957768192004,
 770774730190098433,
 804028724987494402,
 822815182393065476,
 845360182607589376,
 822903103070601216,
 771058757434572800,
 757821654571425792,
 938469204021469184,
 1038380123710545920,
 876064009874747392,
 942925416850907138,
 2266976480,
 786509563927277568,
 1056986636674850816,
 732838734140444676,
 750482565056765952,
 897377572669009920,
 837414074132606978,
 3319800715,
 456361810,
 957638029639913473,
 2197092790,
 3141032242]

In [24]:
mentions_already_processed

[]

In [25]:
# Get neighbors of a node
def add_this_nodes_edgelist(this_acc, curr_edgelist):
    visited.add(this_acc)

    follow_list = []    # list of user id's who show up in first 100 tweets mentioning this user

    # Get the users who mentioned this user and add to edgelist
    request = api.get_users_mentions(this_acc, max_results=100, 
                                     expansions='author_id', 
                                    )
    data = request.json()

    # for each mentioning the originating account, record account id, 
    try:
        for acc in data['data']:
            follow_list.append(acc['author_id'])

    # Catch errors if not authorized to view user
    except Exception as e:
        print('Exception: issue retrieving follow lists for this account -- Skipping.')
        return curr_edgelist
        
    # aggregate # mentions for user as edge weights
    weighted_follow_list = [[x,follow_list.count(x)] for x in set(follow_list)]
    edges_to_add = pd.DataFrame(weighted_follow_list).rename(columns={
                                                                0: 'originating_id',
                                                                1: 'edge_weight'
                                                            })
    edges_to_add['receiving_id'] = this_acc
    curr_edgelist = pd.concat([curr_edgelist, edges_to_add])

    return curr_edgelist

In [26]:
def transition_to_next_separation_degree(curr_edgelist, degrees_of_separation):
    try:
       # create list of all neighbors in curr_edgelist df
        unvisited_neighbors = list(curr_edgelist.originating_id.append(curr_edgelist.receiving_id).unique())

        # if any nodes are in visited set, remove
        [unvisited_neighbors.remove(i) for i in unvisited_neighbors if i in visited]

        print()
        print(f'Current Number of edges from starting node {degrees_of_separation}')
        print(f'Num unvisited neighbors: {len(unvisited_neighbors)}')
        print('--------------------')
        degrees_of_separation +=1 
    
        return curr_edgelist, degrees_of_separation, unvisited_neighbors
    
    
    except Exception as e:
        print(e)
        print('--------------------EXCEPTION--------------------')
        return curr_edgelist, degrees_of_separation, []
        
    

In [27]:
from tqdm.notebook import tqdm

In [28]:
# Create Edgelists for each account in the filtered botometer dataset.
cols=['originating_id', 'receiving_id', 'edge_weight']
      
label = 'human'
for starting_acc in follows_processed:
    if starting_acc in mentions_already_processed:
        continue
    print(f'Starting node == {starting_acc}')
    # Initialization code for populating edgelist starting from node specified by "starting_acc"
    edgelist = pd.DataFrame(columns=cols)

    degrees_of_separation = 0

    visited = set()
    visited.add(starting_acc)

    edgelist = add_this_nodes_edgelist(starting_acc, edgelist) # contains neighbors of starting node
    edgelist, degrees_of_separation, unvisited_neighbors = transition_to_next_separation_degree(edgelist, degrees_of_separation)
    
    # iterate thru all unvisited neighbors with 2 degree of separation from starting (labeled) node
    # process df in chunks before concatenating
    while degrees_of_separation <= 1:
        curr_edgelist = pd.DataFrame(columns=cols)
        for i in tqdm(unvisited_neighbors):
            curr_edgelist = add_this_nodes_edgelist(i, curr_edgelist)
            visited.add(i)
        
        curr_edgelist, degrees_of_separation, unvisited_neighbors = transition_to_next_separation_degree(curr_edgelist, degrees_of_separation)
        edgelist = pd.concat([edgelist, curr_edgelist])

    # Save in directory
    fn = 'edgelist_mentions_' + str(starting_acc) + '.pkl'
    path = str('mentions/' + label) + '/' + fn
    edgelist.to_pickle(path)
    
    # Add twitter id to already processed list
    mentions_already_processed.append(starting_acc)

Starting node == 1039491917145432064

Current Number of edges from starting node 0
Num unvisited neighbors: 80
--------------------


  0%|          | 0/80 [00:00<?, ?it/s]

Exception: issue retrieving follow lists for this account -- Skipping.
Exception: issue retrieving follow lists for this account -- Skipping.

Current Number of edges from starting node 1
Num unvisited neighbors: 3907
--------------------
Starting node == 1047707161571328000

Current Number of edges from starting node 0
Num unvisited neighbors: 12
--------------------


  0%|          | 0/12 [00:00<?, ?it/s]


Current Number of edges from starting node 1
Num unvisited neighbors: 332
--------------------
Starting node == 1012481676524576770

Current Number of edges from starting node 0
Num unvisited neighbors: 7
--------------------


...


Current Number of edges from starting node 1
Num unvisited neighbors: 0
--------------------
Starting node == 2197092790

Current Number of edges from starting node 0
Num unvisited neighbors: 73
--------------------


  0%|          | 0/73 [00:00<?, ?it/s]

Exception: issue retrieving follow lists for this account -- Skipping.
Exception: issue retrieving follow lists for this account -- Skipping.
Exception: issue retrieving follow lists for this account -- Skipping.
Exception: issue retrieving follow lists for this account -- Skipping.

Current Number of edges from starting node 1
Num unvisited neighbors: 2693
--------------------
Starting node == 3141032242

Current Number of edges from starting node 0
Num unvisited neighbors: 22
--------------------


  0%|          | 0/22 [00:00<?, ?it/s]

Exception: issue retrieving follow lists for this account -- Skipping.
Exception: issue retrieving follow lists for this account -- Skipping.

Current Number of edges from starting node 1
Num unvisited neighbors: 649
--------------------
